In [1]:
%load_ext autoreload
%autoreload 2

# Query Mongodb

In [2]:
import utils
from pymongo import MongoClient
configs = utils.get_configs("../configs.yaml")

Connect to MongoDB and get the collection

In [3]:
client = MongoClient(configs['MONGODB_HOST'],configs['MONGODB_PORT'])
db = client[configs['MONGODB_DB_NAME']]
collection = db.get_collection(configs['MONGODB_COLLECTION_NAME'])

Media recensioni selezionando per ciascun anno i top 10:

In [4]:
pipeline = [
    {
        '$match': {
            'box_office.season': { '$nin': [2000, 2001, 2002, 2003] }  # Esclude i film degli anni 2000, 2001, 2002 e 2003
        }
    },
    {
        '$sort': { 'ranking': 1 }  # Ordina i film per ranking in ordine crescente
    },
    {
        '$limit': 400  # Prendo solo i primi 200 film quindi i top 10 per ogni anno dal 2004 in poi
    },
    {
        '$group': {
            '_id': None,
            'avg_mymovies': { '$avg': '$review.MyMovies.rating' },
            'avg_comingsoon': { '$avg': '$review.ComingSoon.rating' },
            'avg_imdb': { '$avg': '$review.IMDb.rating' }
        }
    },
    {
        '$project': {
            '_id': 0,
            'avg_mymovies': { '$round': ['$avg_mymovies', 2] },
            'avg_comingsoon': { '$round': ['$avg_comingsoon', 2] },
            'avg_imdb': { '$round': ['$avg_imdb', 2] }
        }
    }   
]

result = collection.aggregate(pipeline)

for doc in result:
    print(f"MyMovies: {doc['avg_mymovies']}, ComingSoon: {doc['avg_comingsoon']}, IMDb: {doc['avg_imdb']}")

MyMovies: 2.74, ComingSoon: 3.85, IMDb: 6.43


Media recensioni selezionando per ciascun anno i bottom 10:

In [5]:
pipeline1 = [
    {
        '$match': {
            'box_office.season': { '$nin': [2000, 2001, 2002, 2003] }  # Esclude i film degli anni 2000, 2001, 2002 e 2003
        }
    },
    {
        '$sort': { 'ranking': -1 }  # Ordina i film per ranking in ordine crescente
    },
    {
        '$limit': 400  # Prendo solo i primi 200 film quindi i top 10 per ogni anno dal 2004 in poi
    },
    {
        '$group': {
            '_id': None,
            'avg_mymovies': { '$avg': '$review.MyMovies.rating' },
            'avg_comingsoon': { '$avg': '$review.ComingSoon.rating' },
            'avg_imdb': { '$avg': '$review.IMDb.rating' }
        }
    },
    {
        '$project': {
            '_id': 0,
            'avg_mymovies': { '$round': ['$avg_mymovies', 2] },
            'avg_comingsoon': { '$round': ['$avg_comingsoon', 2] },
            'avg_imdb': { '$round': ['$avg_imdb', 2] }
        }
    }   
]

result1 = collection.aggregate(pipeline1)

for doc1 in result1:
    print(f"MyMovies: {doc1['avg_mymovies']}, ComingSoon: {doc1['avg_comingsoon']}, IMDb: {doc1['avg_imdb']}")

MyMovies: 2.74, ComingSoon: 3.85, IMDb: 6.43


Media recensioni totale:

In [6]:
pipeline2 = [
    {
        '$group': {
            '_id': None,
            'avg_mymovies': { '$avg': '$review.MyMovies.rating' },
            'avg_comingsoon': { '$avg': '$review.ComingSoon.rating' },
            'avg_imdb': { '$avg': '$review.IMDb.rating' }
        }
    },
    {
        '$project': {
            '_id': 0,
            'avg_mymovies': { '$round': ['$avg_mymovies', 2] },
            'avg_comingsoon': { '$round': ['$avg_comingsoon', 2] },
            'avg_imdb': { '$round': ['$avg_imdb', 2] }
        }
    }   
]

result2 = collection.aggregate(pipeline2)

for doc2 in result2:
    print(f"MyMovies: {doc2['avg_mymovies']}, ComingSoon: {doc2['avg_comingsoon']}, IMDb: {doc2['avg_imdb']}")

MyMovies: 2.83, ComingSoon: 3.84, IMDb: 6.52


Seleziono i 6 generi più frequenti e calcolo il loro incasso medio.

In [7]:
pipeline3 = [
    {
        '$unwind': '$genere'  # Espande l'array dei generi per ciascun film
    },
    {
        '$group': {
            '_id': '$genere',
            'average_box_office': { '$avg': '$box_office.revenue' },
            'count': { '$sum': 1 }
        }
    },
    {
        '$match': {
            'count': { '$gt': 100 }  # Filtra i generi con frequenza maggiore di 100
        }
    },
    {
        '$project': {
            '_id': 0,
            'genere': '$_id',
            'count': 1,
            'average_box_office': { '$round': ['$average_box_office', 1] }
        }
    },
    {
        '$sort': { 'average_box_office': -1 }  # Ordina per incasso medio in ordine decrescente
    }
]
result3 = collection.aggregate(pipeline3)

# Stampa i risultati

for doc in result3:
    print(f"Genere: {doc['genere']}, Incasso medio: {doc['average_box_office']}, Frequenza: {doc['count']}")

Genere: Animazione, Incasso medio: 130138989.4, Frequenza: 214
Genere: Azione, Incasso medio: 127643513.0, Frequenza: 374
Genere: Commedia, Incasso medio: 59821164.8, Frequenza: 419
Genere: Horror, Incasso medio: 53307530.4, Frequenza: 172
Genere: Thriller, Incasso medio: 50782694.9, Frequenza: 119
Genere: Drammatico, Incasso medio: 43768377.5, Frequenza: 308


Seleziono i primi 10 registi più frequenti e calcolo l'incasso medio

In [8]:
pipeline4 = [
    {
        '$group': {
            '_id': '$register',  # Raggruppa per regista
            'film_count': { '$sum': 1 },  # Conta il numero di film per ciascun regista
            'average_box_office': { '$avg': '$box_office.revenue' }  # Calcola l'incasso medio
        }
    },
    {
        '$sort': { 'film_count': -1 }  # Ordina per numero di film in ordine decrescente
    },
    {
        '$limit': 10  # Limita ai 10 registi più frequenti
    },
    {
        '$project': {
            '_id': 0,
            'register': '$_id',
            'film_count': 1,
            'average_box_office': { '$round': ['$average_box_office', 1] }  # Arrotonda l'incasso medio a una cifra decimale
        }
    }
]

result4 = collection.aggregate(pipeline4)

# Stampa i risultati
for doc in result4:
    print(f"Regista: {doc['register']}, Numero di film: {doc['film_count']}, Incasso medio: {doc['average_box_office']}")

Regista: Steven Spielberg, Numero di film: 16, Incasso medio: 104800670.5
Regista: Clint Eastwood, Numero di film: 14, Incasso medio: 78903756.4
Regista: Ridley Scott, Numero di film: 12, Incasso medio: 80358971.2
Regista: M. Night Shyamalan, Numero di film: 10, Incasso medio: 80591666.1
Regista: Shawn Levy, Numero di film: 10, Incasso medio: 116387148.7
Regista: Steven Soderbergh, Numero di film: 10, Incasso medio: 57732247.3
Regista: Michael Bay, Numero di film: 9, Incasso medio: 178607831.7
Regista: Peter Jackson, Numero di film: 9, Incasso medio: 235784008.0
Regista: Tim Burton, Numero di film: 9, Incasso medio: 121847492.6
Regista: Francis Lawrence, Numero di film: 8, Incasso medio: 205674189.2


Seleziono i 10 film con botteghino più alto e la relativa stagione d'incasso

In [9]:
pipeline5 = [
    {
        '$sort': { 'box_office.revenue': -1 }  # Ordina i film per incasso in ordine decrescente
    },
    {
        '$limit': 10  # Limita ai primi 10 film
    },
    {
        '$project': {
            '_id': 0,
            'title': 1,
            'box_office.revenue': 1,
            'box_office.season': 1  # Include la stagione di produzione
        }
    }
]

result5 = collection.aggregate(pipeline5)
# Stampa i risultati
for doc in result5:
    print(f"Titolo: {doc['title']}, Incasso: {doc['box_office']['revenue']}, Stagione: {doc['box_office']['season']}")

Titolo: Star Wars: Episodio VII - Il risveglio della forza, Incasso: 935518389, Stagione: 2015
Titolo: Avengers: Endgame, Incasso: 858365685, Stagione: 2019
Titolo: Spider-Man - No Way Home, Incasso: 804395488, Stagione: 2021
Titolo: Avatar, Incasso: 749446000, Stagione: 2022
Titolo: Top Gun - Maverick, Incasso: 718519000, Stagione: 2022
Titolo: Black Panther, Incasso: 700004026, Stagione: 2018
Titolo: Avatar - La via dell'acqua, Incasso: 684060555, Stagione: 2022
Titolo: Avengers: Infinity War, Incasso: 678807703, Stagione: 2018
Titolo: Titanic, Incasso: 659328801, Stagione: 2023
Titolo: Jurassic World, Incasso: 652175130, Stagione: 2015


In [10]:
client.close()